## 1. Определение языка записи.

Даны три файла с отрывком из романа Дж. Р. Р. Толкиена "Властелин колец". Про них известно, что слова каждого конкретного текста (за исключением, быть может, небольшого числа шумов) принадлежат некоторому ОДНОМУ языку. Необходимо определить языки каждого текста, затем объединить их в один датасет и написать классификатор, определяющий по строке язык текста, из которого она была взята.

Предварительно было проведено исследование по определению языка каждого символа из текста. Для этого символы были разбиты на характерные группы в соответствии с базой https://unicode-table.com/. При исследовании таблица символов ниже выводилась полностью, вывод убран для удобства чтения.

In [34]:
lts = set()
texts = []
for i in range(2, 5):
    name = str(i) + '.tmp'
    with open(name, encoding='utf-8') as fin:
        texts.append(fin.readlines())
        lts.update({j for i in texts[-1] for j in i})
        
lts = [(i, hex(ord(i))) for i in sorted(list(lts))]
print(lts[: 20])
print(lts[-20: ])
# for i in range(0, len(lts), 6):
#     print(lts[i: i + 6])

[('\x00', '0x0'), ('\n', '0xa'), (' ', '0x20'), ('!', '0x21'), ('"', '0x22'), ('#', '0x23'), ("'", '0x27'), ('(', '0x28'), (')', '0x29'), (',', '0x2c'), ('-', '0x2d'), ('.', '0x2e'), ('0', '0x30'), ('1', '0x31'), ('2', '0x32'), ('3', '0x33'), ('4', '0x34'), ('5', '0x35'), ('6', '0x36'), ('7', '0x37')]
[('默', '0x9ed8'), ('黙', '0x9ed9'), ('點', '0x9ede'), ('黨', '0x9ee8'), ('黯', '0x9eef'), ('鼓', '0x9f13'), ('鼻', '0x9f3b'), ('鼾', '0x9f3e'), ('齒', '0x9f52'), ('齡', '0x9f61'), ('齢', '0x9f62'), ('龍', '0x9f8d'), ('龐', '0x9f90'), ('！', '0xff01'), ('＃', '0xff03'), ('（', '0xff08'), ('）', '0xff09'), ('，', '0xff0c'), ('：', '0xff1a'), ('？', '0xff1f')]


### Характерные группы символов:
    1. Управляющие символы(0x0000-0x001F). В нашем случае их всего два: нуль-символ и символ перевода строки. Они не влияют на классификацию, поэтому их можно смело пропускать.
    2. Основная латинница(0x0020—0x007F). Содержит также графические знаки. В нашем датасете соответствует лишь английскому языку, однако арабские цифры (в силу присутствия их во всех текстах) мы будем относить ко всем трем языкам. Если в английском тексте встретился хотя бы один иероглиф, считаем, что язык записи не английский.
    3. Знаки пунктуации(0x2000—0x206F). Будем относить к английскому языку, так как для языков восточной группы существуют свои знаки препинания.
    4. Символы и пунктуация ККЯ(0x3000—0x303F). Используются в китайском и японском языках.
    5. Катакана и хирагана(0x3040—0x30FF). Используются в японском языке.
    6. Унифицированные иероглифы ККЯ(0x4E00—0x9FFF). Используются в основном в китайском языке (однако, если в тексте встречаются наряду с катаканой и хираганой, это японский язык).
    7. Полуширинные и полноширинные формы(0xFF00—0xFFEF). Используются в китайском и японском языках.

Таким образом, язык очередной записи определяется по utf-кодам символов, в него входящим: достаточно посмотреть, символов какой характерной группы в ней [записи] больше.

Как видно, присутствуют символы лишь из китайского, японского и английского языков. Посмотрим на тексты внимательнее:

In [35]:
for text in texts:
    for record in text[: 10]:
        print(record, end='')
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    for record in text[-10: ]:
        print(record, end='')
    print('\n############################\n')

Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第二十一歲
                                        生日，霍比頓人的話語和興奮也很多。 ＃＃＃＃Bilbo非常富有，非常奇特，自從他的驚人的失踪和意想不
            到的回歸以來，一直是希雷六十年的奇蹟。他從旅行中帶回來的財富現在已經成為一個地方的傳說，無論老民間說
什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力來驚嘆。
              時間穿了，但似乎對Baggins先生影響不大。在九十年代，他和五十歲時一樣。在九十九歲的時候，他們開
              始把他稱得上是保存完好的，但是不會改變的。有一些人搖頭，認為這太好了，任何人都應該擁有（顯然）永久的
青年以及（著名地）無窮無盡的財富似乎是不公平的。他們說：“這將需要支付。” “這不是自然的，麻煩會來
  的！”＃＃但是到目前為止，麻煩沒有到來。正如巴金斯先生慷慨解囊，大多數人都願意赦免他的遺體和幸運。他
仍然與親屬（除了當然是薩克維爾 - 巴金斯）的訪問條件，他在貧窮和不重要的家庭的愛好中有許多忠實的崇
      拜者。但他沒有親密的朋友，直到他的一些年輕的表親才開始長大。 ＃＃這些最老的，Bilbo的最愛，是年
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    變得有霧的時候，我遇到了，然後爬上了庫存，看看你是否在任何溝渠裡摔倒了。但是，如果我知道你來了哪一種
方式，我會很幸福。你在哪裡找到他們，馬格先生？在你的鴨池裡？“＃＃”不，我抓住了，“農夫說，”幾乎把
我的狗放在了但他們會告訴你所有的故事，我毫不懷疑。現在，如果你原諒我，弗里多先生和弗洛多先生，我最好
轉回家。馬格太太會很擔心，晚上變得越來越厚了。“＃他把車子背上車道，把車轉過來。 “嗯，晚安給你們，
  ”他說。 “這是一個奇怪的一天，沒有錯誤。但一切順利結束;儘管也許我們不應該說，直到我們達到自己的門
    。我不會否認現在我會很高興，“他點了燈籠起床。突然，他從座位下面製作了一個大籃子。 “我幾乎忘記了，
  ”他說。 '太太。 Mag got。for。。。。。。。。。。。。。。。。。。。。。。。。。。。。。
    

Как видно, первый текст в основном состоит из сложных и массивных "квадратных" символов. Эти признаки отличают китайскую письменность. Второй текст написан, очевидно, на английском языке (целиком, поэтому при наличии иероглифов запись точно не принадлежит английскому языку). В третьем тексте преобладают простые изящные символы катаканы и хираганы(слегка разбавленные кандзи) - явный признак японского языка.

Напрашивается тривиальный (и, тем не менее, точный) алгоритм решения задачи: переберем все записи из датасета, для каждой посчитаем, сколько символов из неё принадлежит каждому из трех языков, и определим язык записи как argmax последнего. Служебные символы и арабские цифры присутствуют во всех текстах, а потому не влияют на результат.

In [36]:
def predict(text):        
    for record in text:
        langs = [[0, i] for i in range(3)]
        for i in record:
            code = ord(i)
            if code < 0x0020 or i.isdigit():
                continue
                
            if 0x0020 <= code <= 0x007F or 0x2000 <= code <= 0x206F:
                langs[1][0] += 1
            elif 0x3040 <= code <= 0x30FF:
                langs[2][0] += 1
            elif 0x4E00 <= code <= 0x9FFF:
                langs[0][0] += 1
            else:
                langs[0][0] += 1
                langs[2][0] += 1
            
        if langs[0][0] or langs[2][0]:
            langs[1][0] = 0
            
        langs.sort()
        yield langs[2][1]

In [37]:
# preparing data

text, labels = [], []
for i in range(3):
    text.extend(texts[i])
    labels.extend([i] * len(texts[i]))

In [38]:
# full accuracy
from sklearn.metrics import accuracy_score

def accuracy(labels, pred):
    s = 0
    for i in range(len(labels)):
        if labels[i] == pred[i]:
            s += 1
    return s / len(labels)

pred = list(predict(text))
print('Accuracy:', accuracy_score(labels, pred))

Accuracy: 0.991494388659


In [39]:
# KFold
import numpy as np
from sklearn.cross_validation import KFold
from tqdm import tqdm

text = np.array(text)
labels = np.array(labels)

splits = KFold(len(text), n_folds=10, shuffle=True)
accs = []
for train_index, test_index in tqdm(list(splits)):
    x_test, y_test = text[test_index], labels[test_index]
    
    pred = np.array(list(predict(x_test)))
    accs.append(accuracy_score(y_test, pred))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 29.93it/s]


In [40]:
print(accs)

[0.99291617473435656, 0.98701298701298701, 0.99645808736717822, 0.98937426210153478, 0.99173553719008267, 0.99527186761229314, 0.98699763593380618, 0.99054373522458627, 0.99290780141843971, 0.99172576832151305]


## 2. Кодирование текста

Дана строка, состоящая из двухбайтных символов. Необходимо написать алгоритм кодирования и декодирования строки с помощью строки 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/' путем выделения шестибитных индексов из двоичных кодов исходных символов (подробнее - в problems2.pdf). 

In [41]:
import string
import base64

base = string.ascii_uppercase + string.ascii_lowercase + string.digits + '+/'
idx = {base[i]: i for i in range(len(base))}

In [77]:
def encode(text):
    if text == '':
        return ''
    
    if len(text) < 3:
        offset = len(text)
        code = []
        
        block = ord(text[0])
        if offset > 1:
            block <<= 16
            block += ord(text[1])
        
        while block:
            code.append(base[block & ((1 << 6) - 1)])
            block >>= 6
            
        code = [chr(offset + 2)] + list(reversed(code))
        return ''.join(code)
    
    offset = len(text) % 3
    code = [chr(offset)]
    if offset:
        text += '\x00' * (3 - offset)
    
    for i in range(0, len(text), 3):
        block = ord(text[i])
        for j in range(1, 3):
            block <<= 16
            block += ord(text[i + j])
        
        tmp = []
        for j in range(8):
            tmp.append(base[block & ((1 << 6) - 1)])
            block >>= 6
            
        code.extend(reversed(tmp))
        
    return ''.join(code)
    
def decode(code):
    if code == '':
        return ''
    
    offset = ord(code[0])
    
    if offset in {3, 4}:
        block = idx[code[1]]
        for i in code[2: ]:
            block <<= 6
            block += idx[i]
        
        text = []
        while block > 0:
            text.append(chr(block & ((1 << 16) - 1)))
            block >>= 16
        
        text.reverse()
        
        return ''.join(text)
    
    text = []
    for i in range(1, len(code), 8):
        block = idx[code[i]]
        for j in range(1, 8):
            block <<= 6
            block += idx[code[i + j]]
        
        tmp = []
        for j in range(3):
            tmp.append(chr(block & ((1 << 16) - 1)))
            block >>= 16
        
        text.extend(reversed(tmp))
    
    if offset:
        for j in range(3 - offset):
            text.pop()
    
    return ''.join(text)

In [78]:
import sys

for txt in texts:
    for rec in txt[: 3]:
        code = encode(rec)
        code64 = base64.b64encode(bytes(rec, encoding='utf-8'))
        
        t = decode(code)

        print(rec, end='')
        print(code)
        print(code64)
        print(sys.getsizeof(code64) / sys.getsizeof(code))
        print(t)
        print('\n~~~~~~~~~~~~~~~~~~~~\n')
    
    print('\n###########################')
    print('###########################\n')

Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第二十一歲
 AEIAYQBnACAARQBuAGRRSHUfdoQAQgBpAGwAYgBvACAAQgBhAGcAZwBpAG4Ac1FIdR9bo14D/wxO1k4NTkVcB07lcnlSJXaEjx1xTHaEgFpnA2F2eV1O1naEeyxOjFNBTgBrcgAK
b'QmFnIEVuZOWFiOeUn+eahEJpbGJvIEJhZ2dpbnPlhYjnlJ/lrqPluIPvvIzku5bkuI3kuYXlsIfku6XnibnliKXnmoTovJ3nhYznmoTogZrmnIPmhbbnpZ3ku5bnmoTnrKzkuozljYHkuIDmrbIK'
0.9731182795698925
Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第二十一歲


~~~~~~~~~~~~~~~~~~~~

                                        生日，霍比頓人的話語和興奮也很多。 ＃＃＃＃Bilbo非常富有，非常奇特，自從他的驚人的失踪和意想不
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAB1H2Xl/wyXDWvUmBNOunaEinGKnlSMgghZbk5fX4hZGjACACD/A/8D/wP/AwBCAGkAbABiAG+XXl44W8xnCf8Ml15eOFlHcnn/DIHqX55O1naEmlpOunaEWTGOKlSMYQ9g804NAAoAAAAA
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAOeUn+aXpe+8jOmcjeavlOmgk+S6uueahOipseiqnuWSjOiIiOWlruS5n+W+iOWkmuOAgiDvvIPvvIPvvIPvvINCaWxib+mdnuW4uOWvjOacie+8jOmdnuW4uOWlh+eJue+8jOiHquW+nuS7lueahOmpmuS6uueahOWksei4qu

In [79]:
import numpy as np

decodes = []
for rec in text:
    code = encode(rec)
    t = decode(code)
    decodes.append(t)
    
decodes = np.array(decodes)
text = np.array(text)
print(np.where(decodes != text))

(array([], dtype=int64),)


Как видно, на тестовых данных алгоритм работает корректно. Посмотрим, что с эффективностью:

In [80]:
coeffs = []
for txt in texts:
    tmp = []
    for rec in txt:
        code = encode(rec)
        code64 = base64.b64encode(bytes(rec, encoding='utf-8'))
        tmp.append(sys.getsizeof(code64) / sys.getsizeof(code))
    coeffs.append(tmp)
        
for i in range(3):
    print(np.mean(coeffs[i]))

1.1372894749
0.57477644674
1.17447692962


Как видно, наше кодирование проигрывает при шифровании текстов, состоящих из символов с небольшим (меньше двух байт) кодом, например, при шифровании английской речи. С другой стороны, при шифровании текстов из двухбайтовых символов наблюдается выигрыш. Если писать не наивный оптимизированный алгоритм, использующий особенности юникода и нетривиальные идеи для хранения, можно получить более практически значимый алгоритм, однако это выходит за рамки данного задания.